In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
device
block_size = 8
batch_size = 4

In [3]:
torch.cuda.get_device_capability()

(7, 5)

In [2]:
with open('dorian picture.txt','r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

The Picture of Dorian Gray

by Oscar Wilde


Contents

 THE PREFACE
 CHAPTER I.
 CHAPTER II.
 CHAPTER III.
 CHAPTER IV.
 CHAPTER V.
 CHAPTER VI.
 CHAPTER VII.
 CHAPTER VIII.
 CHAPTER IX.
 CHAPTER X.



In [6]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', ',', '-', '.', '0', '1', '2', '5', '8', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'É', 'à', 'æ', 'ç', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '\ufeff']


# Lets create an encode and decode function 

Only for the characters that are present in the book.

In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda s2: [int_to_string[c] for c in s2]

data = torch.tensor(encode(text), dtype = torch.long)

In [5]:
data

tensor([80, 33, 48,  ..., 17,  0,  0])

# Get get_batch

In [9]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([ 12986,  23004,  10730, 326890])
inputs:
tensor([[ 1, 60, 48, 41, 60,  1, 59, 55],
        [52, 44,  1, 53, 49, 47, 48, 60],
        [45, 52, 52,  1, 65, 55, 61,  1],
        [ 1, 41, 53,  1, 47, 52, 41, 44]], device='cuda:0')
targets:
tensor([[60, 48, 41, 60,  1, 59, 55, 53],
        [44,  1, 53, 49, 47, 48, 60,  1],
        [52, 52,  1, 65, 55, 61,  1, 63],
        [41, 53,  1, 47, 52, 41, 44,  1]], device='cuda:0')


# Bigram Language Model 

What are we doing here:


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self,index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)

            logits = logits[:, -1, :]
            probs = F.softmax(logits,dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index,index_next),dim=1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)